In [58]:
import os
import json
from time import time
import cv2 as cv
import numpy as np
import pickle as pkl
from glob import glob
import matplotlib.pyplot as plt

from utils import *

In [60]:
%run scripts/all_measures_to_json.py
%run scripts/merge_anns_and_measures.py

In [61]:
PATH = "data"

with open(os.path.join(PATH, "anns_w_measure.json"), "r") as f:
    anns = json.load(f)
    gt_anns_measures = anns["annotations"]
    imgs_data = anns["images"]

In [62]:
PKL_PATH = "/home/lazuli/Desktop/Projetos/PKLS/C3S-Clinker-dataset/Original"
with open(os.path.join(PKL_PATH, "dt_masks.pkl"), "rb") as f:
    all_dt_masks = pkl.load(f)

# preprocess masks dict

filename_and_id = [[x["file_name"], x["id"]] for x in imgs_data]
filename_to_id = dict(filename_and_id)
id_to_filename = dict([x[::-1] for x in filename_and_id])

img_filepaths, masks = zip(*all_dt_masks.items())
img_filenames = [os.path.basename(x) for x in img_filepaths]
img_ids = [filename_to_id[f] for f in img_filenames]

all_dt_masks = dict(zip(img_ids, masks))

In [63]:
IOU_THRESH = 0.5 # threshold used to validate if it's a hit or not

all_eval_data = []
total_fps = 0
total_fns = 0
for img_id, anns_measures in list(gt_anns_measures.items()):
    img_id = int(img_id)
    img_data = imgs_data[img_id]
    height, width = img_data["height"], img_data["width"]
    
    ids = [x["id"] for x in anns_measures]
    gt_poly = [x["segmentation"] for x in anns_measures]
    gt_measures = [x["measure"] for x in anns_measures]
    gt_masks = [poly_to_mask(x, (height, width), True) for x in gt_poly]
    dt_masks = all_dt_masks[img_id]

    ious_mx = all_iou_combinations(gt_masks, dt_masks)
    ious_mx_hit = ious_mx >= IOU_THRESH
    
    hit_sum_gt = ious_mx_hit.sum(axis=1)
    hit_sum_dt = ious_mx_hit.sum(axis=0)
    
    fns = np.where(hit_sum_gt == 0)
    tps = np.where(hit_sum_gt > 0)
    
    # increase total number of false positives and negatives
    total_fps += np.count_nonzero(hit_sum_dt == 0)
    total_fps += np.count_nonzero(hit_sum_gt > 1)
    total_fns += len(fns)

    gt_ious = ious_mx.max(axis=1)
    gt_ious[fns] = -1
    corresp_dt_index = ious_mx.argmax(axis=1)
    corresp_dt_index[fns] = -1
    
    gt_areas = [x.sum() for x in gt_masks]
    dt_areas = [dt_masks[i].sum() if i>=0 else -1 for i in corresp_dt_index]
    
    # append data
    filename = id_to_filename[img_id]
    img_ids = [img_id for _ in anns_measures]
    filenames = [filename for _ in anns_measures]
    data = list(zip(filenames, img_ids, ids, gt_areas, gt_measures, corresp_dt_index, dt_areas, gt_ious))
    all_eval_data.extend(data)
    
df = pd.DataFrame(all_eval_data, columns=[
    "filename", "image_id", "id", "area", "ground-truth_measure", 
    "detected_mask_id", "detected_mask_area", "iou"])

In [65]:
# measure and collect data
from measure_methods import *

name_methods = ["pca", "centroid", "lr", "brute-force"]
methods = [longest_diagonal_pca, longest_diagonal_centroid,
    longest_diagonal_lr, farthest_pair_of_points_brute_force]
kwargs_methods = [{}, {"use_find_peaks": True}, {}, {}]

# creating and initializing new columns
df["find-contour_time"] = -1.0
for method in name_methods:
    df[f"{method}_measure"] = -1.0
    df[f"{method}_time"] = -1.0
    
for i,row in df.iterrows():
    img_id = row["image_id"]
    dt_mask_id = row["detected_mask_id"]
    
    if dt_mask_id < 0: continue
    
    dt_mask = all_dt_masks[img_id][dt_mask_id]
    
    start = time()
    contours,_ = cv.findContours(dt_mask.astype(np.uint8), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
    contour = max(contours, key=lambda x: x.shape[1])
    points = contour.squeeze()
    end = time()
    df.at[i,"find-contour_time"] = end-start
    
    if len(points.shape) != 2: continue
    
    for name, method, kwargs in zip(name_methods, methods, kwargs_methods):        
        start = time()
        _,_,length = method(points, **kwargs)
        end = time()
        df.at[i,f"{name}_time"] = end-start
        df.at[i,f"{name}_measure"] = length

ValueError: `distance` must be greater or equal to 1

In [ ]:
df.iloc[0:10]

In [ ]:
df.to_csv(os.path.join(PATH, "eval_measuring.csv"))
with open(os.path.join(PATH, "eval_data.csv"), "w") as f:
    json.dump({"total_FP": total_fps, "total_FN":total_fns}, f)